# Data Science Final Project - EE 379k - sp18 - 

# Shahshank Kambhampati - skk834, Shrikara Murthy - svm456, Pranav Harathi - , Neil Charles - 

# Job Satisfaction Prediction

## Imports

### All hail lord and savior XGB

In [9]:
import numpy as np
import pandas as pd
import re

import xgboost as xgb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from scipy import stats
from sklearn.linear_model import RidgeCV, LassoCV, LogisticRegression, Ridge, Lasso
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomTreesEmbedding, AdaBoostClassifier, AdaBoostRegressor, RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import make_scorer, roc_auc_score, mean_squared_error, accuracy_score
from sklearn.model_selection import LeaveOneOut, train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE,RFECV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest,  chi2
from sklearn.svm import SVR

%config inlinebackend.figure_format = 'retina' 
%matplotlib inline

# Initial Observations about the Data and Pre-Processing

In [3]:
number = re.compile('[\d,]+')
def get_first_number(val):
    matched = number.match(str(val))
    if matched:
        return int(matched.group())
    else:
        return np.nan

satisfaction_strs = {
    'I love my job': 10,
    'I\'m somewhat satisfied with my job': 7.5,
    'I\'m neither satisfied nor dissatisfied with my job': 5,
    'I\'m somewhat dissatisfied with my job': 2.5,
    'I hate my job': 0,
}

binary_labels = [
    'Lang & Tech',
    'Training & Education',
    'How can companies improve interview process',
    'Why try Stack Overflow Careers',
    'Most important aspect of new job opportunity',
    'Most annoying about job search',
    'Appealing message traits',
    'Most urgent info about job opportunity',
    'Who do you want to communicate with about a new job opportunity',
    'Why use Stack Overflow',
    'Why answer',
    'Source control used',
]

numeric_labels = [
    'Age',
    'Years IT / Programming Experience',
    'Compensation: midpoint'
]

yes_no_labels = [
    'Changed Jobs in last 12 Months'
]


In [31]:
# 2015 preproc
data = pd.read_csv('data/2015.csv')
to_drop = [label for label in data if 'write-in' in label.lower()]

data['Job Satisfaction'] = data['Job Satisfaction']\
                                    .map(satisfaction_strs).astype('float')
data = data[data['Job Satisfaction'].notnull()]
to_drop.append('Country')
to_drop.append('Compensation')

data['Age'] = data['Age'].map(get_first_number).astype('float')
data['gender_M'] = (data['Gender'] == 'Male').astype('int8')
data['gender_F'] = (data['Gender'] == 'Female').astype('int8')
to_drop.append('Gender')
to_drop.append('Prefered Source Control')

bin_labels = [key for key in data if any(label in key for label in binary_labels)]
data[bin_labels] = data[bin_labels].apply(lambda col: col.notnull().astype('int8'))

data[numeric_labels] = data[numeric_labels].applymap(get_first_number)
data[yes_no_labels] = data[yes_no_labels]\
                                .apply(lambda col: col.map({'Yes': 1, 'No': 0}))\
                                .fillna(0)

data.drop(to_drop, axis=1, inplace=True)
data = pd.get_dummies(data)

print 'Columns with NaNs:'
for key in data:
    if data[key].isnull().any():
        print key
data.fillna(data.mean(), inplace=True)
print 'Filling with mean of column'

Columns with NaNs:
Age
Years IT / Programming Experience
Compensation: midpoint
Filling with mean of column


In [32]:
# data contains a DataFrame with no NaNs, all numbers.
# We're trying to predict the "Job Satisfaction" column.

In [33]:
X = data.drop(['Job Satisfaction'], axis=1)
y = data['Job Satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)#stratify=y
scoring = {'mean': make_scorer(mean_squared_error)}

In [34]:
print data.head()

    Age  Years IT / Programming Experience  Current Lang & Tech: Android  \
0  25.0                                2.0                             0   
1  20.0                                1.0                             0   
2  20.0                                1.0                             0   
3  25.0                                6.0                             0   
4  30.0                                2.0                             0   

   Current Lang & Tech: Arduino  Current Lang & Tech: AngularJS  \
0                             0                               0   
1                             0                               0   
2                             0                               0   
3                             0                               0   
4                             0                               0   

   Current Lang & Tech: C  Current Lang & Tech: C++  \
0                       0                         0   
1                       0     

In [ ]:
print data.describe()

In [37]:
for i in data.corr():
    print i

Age
Years IT / Programming Experience
Current Lang & Tech: Android
Current Lang & Tech: Arduino
Current Lang & Tech: AngularJS
Current Lang & Tech: C
Current Lang & Tech: C++
Current Lang & Tech: C++11
Current Lang & Tech: C#
Current Lang & Tech: Cassandra
Current Lang & Tech: CoffeeScript
Current Lang & Tech: Cordova
Current Lang & Tech: Clojure
Current Lang & Tech: Cloud
Current Lang & Tech: Dart
Current Lang & Tech: F#
Current Lang & Tech: Go
Current Lang & Tech: Hadoop
Current Lang & Tech: Haskell
Current Lang & Tech: iOS
Current Lang & Tech: Java
Current Lang & Tech: JavaScript
Current Lang & Tech: LAMP
Current Lang & Tech: Matlab
Current Lang & Tech: MongoDB
Current Lang & Tech: Node.js
Current Lang & Tech: Objective-C
Current Lang & Tech: Perl
Current Lang & Tech: PHP
Current Lang & Tech: Python
Current Lang & Tech: R
Current Lang & Tech: Redis
Current Lang & Tech: Ruby
Current Lang & Tech: Rust
Current Lang & Tech: Salesforce
Current Lang & Tech: Scala
Current Lang & Tech: Shar

In [ ]:
model = xgb.XGBRegressor()
rfe = RFE(model, 100)
rfe = rfe.fit(X,y)
rfe.transform(X_train)
rfe.transform(X_test)

In [18]:
parameters = {
    'n_estimators':[160]
}
modelETC = ExtraTreesRegressor()
gridETC = GridSearchCV(modelETC, param_grid=parameters,scoring = scoring, cv=5, refit = 'mean',verbose=1)
gridETC.fit(X_train,y_train)
print(gridETC.best_params_)
#print(modelETC.feature_importances_)
etc_preds = gridETC.predict(X_test)
print mean_squared_error(y_test,etc_preds)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  8.4min finished


{'n_estimators': 160}


IndexError: too many indices for array

In [19]:
print mean_squared_error(y_test,etc_preds)

8.4194594212


In [15]:
parameters = { #when use hyperthread, xgboost may become slower
              #'objective':['binary:logistic'],
             # 'booster':['dart'],
               'booster':['gbtree'],
              'learning_rate': [.5], #so called `eta` value
              'max_depth': [9],#
              'min_child_weight': [5],#
              'reg_alpha':[.1],
              'silent': [0],
              'subsample': [.7],
              'gamma':[.3],
              'colsample_bytree': [.8],
              'n_estimators': [100], #number of trees, change it to 1000 for better results
              'seed': [1337]
}

In [20]:
xgb_model = xgb.XGBRegressor()
#gridXGB = GridSearchCV(xgb_model, param_grid=parameters,scoring = scoring, cv=5, verbose=2, refit = 'mean')
#gridXGB.fit(X_train, y_train,eval_metric='rmse')
#print(gridXGB.best_params_)
#print(grid.score(X_test, y_test))
#xgb_preds = gridXGB.predict(X_test)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
print mean_squared_error(y_test,xgb_preds)

5.07340855855


In [21]:
for pred in xgb_preds:
    print pred

7.36924
6.62353
8.35021
6.29295
6.0813
6.35723
7.41793
7.99174
9.64845
6.14148
9.8153
7.8068
8.1129
7.77289
7.65919
7.90116
5.49829
4.98862
7.06453
8.43476
7.14964
6.82427
8.28217
6.70608
7.12675
5.57958
8.31992
4.32397
8.83472
7.31375
7.14522
7.01388
7.36496
5.69643
8.6648
3.73134
8.85487
7.9686
7.78287
6.42285
6.1477
4.18053
9.08031
7.14732
7.3288
8.81392
7.04292
8.66282
6.96666
7.43086
8.74762
9.1361
8.97712
7.99763
6.66776
8.33043
8.04744
8.72336
7.71168
9.26748
7.569
8.84615
9.16948
6.85384
6.53246
7.52489
8.82086
8.02662
7.03778
8.52096
6.27912
8.48434
6.56954
6.57167
9.04473
6.22696
7.91215
8.72664
8.73495
7.70561
8.27631
7.29603
3.87241
6.93092
6.1045
7.55898
7.34052
7.32217
8.90193
5.86161
4.51041
7.1011
8.97622
8.956
6.99595
8.52602
7.67594
9.53981
7.38799
7.37253
8.13964
6.3299
6.75734
6.61789
9.18352
8.62922
6.29766
4.4125
9.55628
8.25919
8.80202
5.51069
7.39844
7.507
8.86469
8.96695
7.35516
7.44769
7.54047
7.28765
7.11834
7.77171
7.12657
7.35841
7.2243
8.38763
5.64902
6.08

5.08442
7.7453
7.13739
7.06256
6.91851
8.70361
8.51666
6.19366
7.3062
6.98994
7.09216
8.76801
9.38351
7.23364
7.70949
8.75374
7.59197
7.11048
6.74722
8.47816
7.12799
6.20595
6.92637
8.83703
8.74953
6.43656
8.5424
7.0487
7.59831
8.02738
8.51278
7.768
7.2182
7.38056
8.01293
6.39943
8.19536
8.98611
6.862
7.19492
6.92333
8.11167
9.00454
8.23187
5.28187
7.70314
7.54194
5.30251
9.25749
8.72886
8.14204
7.4033
9.3602
7.28531
7.30647
6.58747
8.80809
7.41785
6.85307
7.1914
7.92875
6.71397
6.85964
6.94839
8.54645
6.46683
8.19434
9.04011
8.21252
8.7293
6.70367
6.98606
8.53443
9.34476
8.36619
8.6419
7.02624
8.05894
6.7874
7.63665
8.53457
7.35143
8.99622
6.16963
6.97079
5.52269
7.57504
8.60011
7.61271
7.22306
8.59814
7.47289
6.64933
4.9192
8.63599
7.22922
5.28717
7.44028
6.21247
7.23369
9.44081
8.97985
8.48564
6.57939
7.5735
4.61481
7.01953
8.6266
9.41694
8.5083
5.57637
4.60079
4.56257
7.08434
8.72029
9.0848
6.74949
8.57921
6.54157
8.31723
8.52955
8.44613
7.89888
7.10832
8.86548
7.88068
6.29242
7.25